In [2]:
# Import statements
import matplotlib.pyplot as plt
from matplotlib import animation

import numpy as np
import pandas as pd
import scipy.special as sp

import datetime
import random
import time

from string import ascii_lowercase

In [3]:
# Simple ascii to number dictionary

nodeDict = {}
i = 0
for c in ascii_lowercase:
    nodeDict[c] = i
    i += 1

In [22]:
# Node class
class Node:
    
    
    # Initialization
    def __init__(self, x_pos, node_id):
        self.chain = []
        self.x_chain = []
        self.x_chain.append(x_pos)
        self.id = node_id
        self.radius = 1
        self.isEvil = False
        
    # Every timestamp, we connect to all nodes nearby
    def connect(self, nodes, timestamp):
        self.chain.append([])
        
        for node in nodes:
            
            # Distance from other node to this node
            # Note that distance (in x_chain is always timestamp + 1 to avoid off-by-one errors)
            dist = (node.x_chain[timestamp] - self.x_chain[timestamp])
            
            # Check if nearby nodes are close enough
            if  abs(dist) <= self.radius and node.id != self.id:
                
                # If the nodes are to this node's right, add them with (self, node)
                if dist > 0:
                    self.chain[timestamp].append([self.id, node.id])

                # Otherwise, add in the order (node, self)
                else:
                    self.chain[timestamp].append([node.id, self.id])
    
    # Move in a random direction
    def move(self):
        self.x_pos = self.x_pos + random.randint(-1, 1)

    # toString method
    def __str__(self):
        return ('ID:%3s Position:%3s Evil:%3s' % (self.id, self.x_chain, self.isEvil))

    
    
# EvilNode, type 1 class
class EvilNode1(Node):

    # Calls the super constructor
    def __init__(self, x_pos, node_id):
        super().__init__(x_pos, node_id)
        self.isEvil = True
        
    # Same as above, except only connects with other evilNodes
    def connect(self, nodes, timestamp):
        self.chain.append([[],[]])
        for node in nodes:

            # Distance from other node to this node
            # Note that distance (in x_chain is always timestamp + 1 to avoid off-by-one errors)
            dist = (node.x_chain[timestamp] - self.x_chain[timestamp])
            
            # Check if nearby nodes are close enough
            if  abs(dist) <= self.radius and node.id != self.id and node.isEvil == True:
                
                # If the nodes are to this node's right, add them with (self, node)
                if dist > 0:
                    self.chain[timestamp].append([self.id, node.id])

                # Otherwise, add in the order (node, self)
                else:
                    self.chain[timestamp].append([node.id, self.id])
    

In [23]:
# Initialize array of Nodes
nodeList = []

for c in ascii_lowercase[0:5]:
    
    # All honest Nodes for now
    if ((random.randint(0,10) % 3) == 0):
        
        node1 = Node(random.randint(0, 5), c)
        nodeList.append(node1)
    else:
        node1 = EvilNode1(random.randint(0, 5), c)
        nodeList.append(node1)

In [24]:
for node in nodeList:
    node.connect(nodeList, 0)
    
for node in nodeList:
    print(node)

ID:  a Position:[0] Evil:False
ID:  b Position:[4] Evil:False
ID:  c Position:[0] Evil:False
ID:  d Position:[0] Evil:False
ID:  e Position:[1] Evil:True


In [32]:
# Assumes that nodes accurately report their own location
# Checks all the neighbors to the left and right that the node reports
# Checks all that the left neighbors report each other as neighbors (and also report the node in question)
# Repeat for the right

# Queries the "chain" of nodes to see if a node was at (X, T)
def queryNode(time, node_id, location, nodeList):
    
    # Grab the appropriate node
    node = nodeList[nodeDict[node_id]]
    
    # Check to see that the node reports it is where it is at
    if (node.x_chain[time] != location):
        print("ERROR: Node does not self-report to be at specified location")
        return False
    
    else:
        # Initializing variables
        total_witnesses_count = len(node.chain[time])
        unseen_witnesses_count = 0
        
        total_left_unseen_neighbors_count = 0
        total_right_unseen_neighbors_count = 0
        
        left_unseen_neighbors_count = 0
        right_unseen_neighbors_count = 0
        
        left_nodes = []
        right_nodes = []
        
        # Split into left and right neighbors
        partitionNeighbors(node.chain[time], left_nodes, right_nodes, node_id)
        
        # DEBUGGING
        print('Left neighbors: %5s | Right neighbors: %5s' % (left_nodes, right_nodes))
        
        left_unseen_neighbors_count = len(left_nodes)
        right_unseen_neighbors_count = len(right_nodes)
        
        # Check to see if all nodes saw our node
        unseen_witnesses_count = countWitnesses(left_nodes+right_nodes, node_id, time, nodeList)

        # DEBUGGING
        actual_witnesses_num = total_witnesses_count - unseen_witnesses_count
        print('Number of witnesses: %s' % (actual_witnesses_num))
        
        # Check to see if all right nodes saw our node
        left_unseen_neighbors_count = countNeighbors(left_nodes, time, nodeList)
        
        # DEBUGGING
        print('Number of left neighbors who saw one another: %s' %(left_unseen_neighbors_count))
        
        # Checks if all right nodes saw each other (this can blow up quickly in time)
        right_unseen_neighbors_count = countNeighbors(right_nodes, time, nodeList)
        
        # DEBUGGING
        print('Number of right neighbors who saw one another: %s' %(right_unseen_neighbors_count))
        
        # Final print statement
        print('node \'%s\' has %s witnesses | \
%s left neighbor verifications | \
%s right neighbor verifications' 
                      % (node_id, actual_witnesses_num, left_unseen_neighbors_count, 
                         right_unseen_neighbors_count))
        
        # Do the final check to see if the number of witnesses and neighbor mutual chains is > 0.5
        if (unseen_witnesses_count > 0.5 * total_witnesses_count):
            
            # Calculate 50% threshold for neighbors
            left_threshold = 0.5 * sp.comb(total_left_unseen_neighbors_count, 2)
            right_threshold = 0.5 * sp.comb(total_right_unseen_neighbors_count, 2)
            
            # Only print if the threshold is met
            if (left_unseen_neighbors_count > left_threshold and right_unseen_neighbors_count > right_threshold):
                
                return True
            
            else:
                return False
        
        else:
            return False

    
    
# Helper function that splits up into left and right neighbors
def partitionNeighbors(timeChain, left_nodes, right_nodes, node_id):
    for link in timeChain:
        if (link[0] == node_id):
            right_nodes.append(link[1])
        else:
            left_nodes.append(link[0])



# Checks to see how many of the neighbors saw the node
def countWitnesses(neighbor_nodes, node_id, time, nodeList):
    
    seen_witnesses_count = 0
    
    for neighbor in neighbor_nodes:
    
        # Iterate through all of the neighbor's chains at the time
        for link in nodeList[nodeDict[neighbor]].chain[time]:
            
            # If it saw our node, break out and add 1
            if node_id in link:
                seen_witnesses_count = seen_witnesses_count + 1
                
                # DEBUGGING
                print('Saw node \'%s\' in link:%s' % (node_id, link))
                break        
            
    # Returns the number of nodes which didn't see our node
    return (len(neighbor_nodes) - seen_witnesses_count)



# Checks the neighbors to see if they saw one another
def countNeighbors(neighbor_list, time, nodeList):
    
    unseen_neighbors_count = 0
    
    for neighbor in neighbor_list:
        
        # Grab the list of all neighbors except for this one
        all_other_neighbors = set([x for x in neighbor_list if x != neighbor])
        
        # DEBUGGING
        print('All neighbors except \'%s\':%3s' % (neighbor, all_other_neighbors))
        
        all_elements_in_chain = set([item for sublist in nodeList[nodeDict[neighbor]].chain[time] for item in sublist])
        
        for neighbor in all_other_neighbors:
            if (neighbor not in all_elements_in_chain):
                seen_neighbors_count += 1
                     
    print(unseen_neighbors_count)
    
    return unseen_neighbors_count

In [33]:
queryNode(0, 'd', 0, nodeList)

Left neighbors: ['a', 'c'] | Right neighbors: ['e']
Saw node 'd' in link:['d', 'a']
Saw node 'd' in link:['d', 'c']
Number of witnesses: 2
All neighbors except 'a':{'c'}
All neighbors except 'c':{'a'}
0
Number of left neighbors who saw one another: 0
All neighbors except 'e':set()
0
Number of right neighbors who saw one another: 0
node 'd' has 2 witnesses | 0 left neighbor verifications | 0 right neighbor verifications


False